# Folium Explore
- Using `chloropleth`
- https://towardsdatascience.com/choropleth-maps-with-folium-1a5b8bcdd392
- https://georgetsilva.github.io/posts/mapping-points-with-folium/

In [30]:
import pandas as pd 
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [31]:
import folium
from folium.features import GeoJsonPopup, GeoJsonTooltip

In [113]:
url = (
    "https://raw.githubusercontent.com/python-visualization/folium/master/examples/data"
)
state_geo = f"{url}/us-states.json"
state_unemployment = f"{url}/US_Unemployment_Oct2012.csv"
state_data = pd.read_csv(state_unemployment)
state_gdf = gpd.read_file(state_geo).set_index("id").join(state_data.set_index("State")).reset_index()

In [114]:
import matplotlib
from matplotlib import cm


def _scale_values(values: np.ndarray) -> np.ndarray:
    amin = values.min()
    amax = values.max()
    return (values - amin) / (amax - amin)


def assign_color_property(gdf: gpd.GeoDataFrame, column_str: str, cmap: str = "viridis") -> gpd.GeoDataFrame:
    values = gdf[column_str]
    colormap = cm.get_cmap(cmap, len(values))
    # normalize values
    values = _scale_values(values)

    colors = [colormap(v) for v in values]
    hexes = [matplotlib.colors.rgb2hex(c) for c in colors]
    gdf["rgba"] = colors
    gdf["hex"] = hexes
    return gdf

In [89]:
gdf = assign_color_property(gdf, "Unemployment").head()

In [91]:
# https://stackoverflow.com/questions/35516318/plot-colored-polygons-with-geodataframe-in-folium
m = folium.Map(location=[48, -102], zoom_start=3)

tooltip = GeoJsonTooltip(
    fields=["name", "Unemployment"],
    aliases=["State:", "2015 Unemployment (%):"],
    localize=True,
    sticky=False,
    labels=True,
    style="""
        background-color: #F0EFEF;
        border: 2px solid black;
        border-radius: 3px;
        box-shadow: 3px;
    """,
    max_width=800,
)

folium.GeoJson(
    data=state_gdf,
    tooltip=tooltip,
    # style the polygons as a chloropleth
    style_function=lambda feature: {
        'fillColor': feature['properties']['hex'],
        'color' : feature['properties']['hex'],
        'weight' : 1,
        'fillOpacity' : 0.5,
    }
).add_to(m)


folium.LayerControl().add_to(m)

m

# Folium Plot

In [118]:
import pickle 
from pathlib import Path
from typing import Optional

metric_gdf = pickle.load(Path("/cats/datastore/data/RUNOFF/complexity_metrics.gdf").open("rb"))

In [119]:
import matplotlib
from matplotlib import cm


def _scale_values(values: np.ndarray) -> np.ndarray:
    amin = values.min()
    amax = values.max()
    numerator = (values - amin)
    denominator = (amax - amin)
    return (values - amin) / (amax - amin)


def assign_color_property(gdf: gpd.GeoDataFrame, column_str: str, cmap: str = "viridis_r", rescale_vmin_vmax: bool = False, vmin: Optional[float] = 0.7, vmax: Optional[float] = 1.0) -> gpd.GeoDataFrame:
    values = gdf[column_str]
    colormap = cm.get_cmap(cmap, len(values))
    
    if rescale_vmin_vmax:
        if vmin is None:
            vmin = np.quantile(values, 0.25)
        if vmax is None:
            vmax = np.quantile(values, 0.75)

        values[values < vmin] = vmin
        values[values > vmax] = vmax
    # values mapped to 0, 1 scale
    values = _scale_values(values)

    colors = [colormap(v) for v in values]
    hexes = [matplotlib.colors.rgb2hex(c) for c in colors]
    gdf["rgba"] = colors
    gdf["hex"] = hexes
    return gdf

In [121]:
def make_tooltip_str(row: pd.Series):
    from pandas.api.types import is_numeric_dtype
    from pprint import pformat
    
    tooltip_keys = [c for c in row.index if not c in ["geometry", "hex", "rgba"]]
    numeric_keys = np.array(tooltip_keys)[[isinstance(c, float) for c in row[tooltip_keys]]]
    row[numeric_keys] = [np.round(row[numeric_key], 2) for numeric_key in numeric_keys]
    return pformat(row[tooltip_keys].to_dict()).replace("{", "").replace("}", "").replace("\n", "<br>").replace("'", "")


def make_tooltip(row):
    tooltip_str = make_tooltip_str(row)
    return folium.Tooltip(
        tooltip_str,
        style=(''),
    )

In [129]:
metric_gdf = assign_color_property(metric_gdf, "hs_004_0303_181524", vmin=0.7, vmax=1, rescale_vmin_vmax=True)

/cats/datastore/data/minconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/cats/datastore/data/minconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [130]:
# scale = 1.5
# width=300*scale, height=500*scale,
m = folium.Map(location=[55., -4.], zoom_start=6, tiles="cartodbpositron")

for ix, row in metric_gdf.iterrows():
    lat, lon = row["geometry"].y, row["geometry"].x
    folium.Circle(
        location=[lat, lon],
        radius=20,
        fill=True,
        color=row["hex"],
        tooltip=make_tooltip(row),
        popup=make_tooltip_str(row),
        #fill_opacity=0.7
    ).add_to(m)


folium.LayerControl().add_to(m)

m